<a href="https://colab.research.google.com/github/umji4500/Master-Thesis/blob/main/7_day_SSO_sim_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === Comprehensive SSO Sweep (7-Day Data Generation) ===
# Description: This script runs a full factorial sweep for SSO configurations over a 7-day
#              period and saves the complete raw results to a single CSV file.

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta, time, timezone
import time as py_time

print("--- Starting Comprehensive SSO Sweep (7-Day Data Generation) ---")

# ==============================================================================
# 1. CORE CONSTANTS AND CONFIGURATION
# ==============================================================================

# --- Physical and Orbital Constants (SI units) ---
MU = 3.986004418e14      # Earth's gravitational parameter, m^3/s^2
R_EARTH = 6371e3          # Earth radius, m
J2 = 1.08263e-3           # Earth’s J2 perturbation coefficient
OMEGA_DOT_SUN = 1.99106e-7 # Desired RAAN drift for SSO, rad/s
OMEGA_EARTH = 7.2921159e-5  # Earth rotation rate, rad/s

# --- Simulation Time Parameters ---
EPOCH = datetime(2025, 3, 20, 0, 0, 0)
TIME_STEP = 10
SIM_DURATION = 7 * 24 * 3600

# --- Ground Target (Incheon Airport) ---
TARGET_LAT = np.radians(37.4602)
TARGET_LON = np.radians(126.4407)
MIN_ELEVATION = np.radians(20)

# --- Save Directory & Filename ---
save_dir = '/content/drive/MyDrive/ThesisData'
output_filename = "All_SSO_Sweep_Raw_Results_7day.csv"
output_path = os.path.join(save_dir, output_filename)
print(f"Output will be saved to: {output_path}")
try:
    os.makedirs(save_dir, exist_ok=True)
except Exception as e:
    print(f"⚠️ Warning: Could not create or access save directory: {e}")

# --- CORRECTED: Define KST Sunrise/Sunset Times for 7 days ---
daily_sun_times_kst = {
    '2025-03-20': (time(6, 36), time(18, 43)),
    '2025-03-21': (time(6, 34), time(18, 44)),
    '2025-03-22': (time(6, 32), time(18, 45)),
    '2025-03-23': (time(6, 31), time(18, 46)),
    '2025-03-24': (time(6, 29), time(18, 47)),
    '2025-03-25': (time(6, 28), time(18, 48)),
    '2025-03-26': (time(6, 26), time(18, 49)),
    '2025-03-27': (time(6, 25), time(18, 50)),
}
print(f"Using daily KST sunrise/sunset times for {len(daily_sun_times_kst)} days.")
kst_offset = timedelta(hours=9)

# ==============================================================================
# 2. HELPER FUNCTIONS (Integrated and Corrected)
# ==============================================================================

def inclination_for_sso(a):
    n = np.sqrt(MU / a**3)
    term = (-2 * OMEGA_DOT_SUN * a**2) / (3 * J2 * R_EARTH**2 * n)
    return np.arccos(np.clip(term, -1.0, 1.0))

def solve_kepler(M, e, tol=1e-8):
    E = M if e < 0.8 else np.pi
    for _ in range(100):
        f = E - e * np.sin(E) - M
        f_prime = 1 - e * np.cos(E)
        if abs(f_prime) < 1e-12: break
        E -= f / f_prime
        if abs(f) < tol: break
    return E

def propagate_orbit(a, e, inc, RAAN, AOP, M0, duration_sec, dt_sec):
    n = np.sqrt(MU / a**3)
    if dt_sec <= 0: return None, None
    steps = int(duration_sec / dt_sec)
    if steps <= 0: return np.array([]), np.array([])
    positions, times = [], []
    p = a * (1 - e**2)
    Omega_dot = -1.5 * J2 * (R_EARTH**2) * n * np.cos(inc) / (p**2)
    omega_dot = 0.75 * J2 * (R_EARTH**2) * n * (5 * np.cos(inc)**2 - 1) / (p**2)
    M_dot = n + 0.75 * J2 * (R_EARTH**2) * n * (3 * np.cos(inc)**2 - 1) / (p**2)
    for step in range(steps):
        t = step * dt_sec
        RAAN_t, AOP_t, M_t = RAAN + Omega_dot * t, AOP + omega_dot * t, M0 + M_dot * t
        E = solve_kepler(M_t % (2 * np.pi), e)
        TA = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2), np.sqrt(1 - e) * np.cos(E / 2))
        r_mag = a * (1 - e * np.cos(E))
        r_pf = np.array([r_mag * np.cos(TA), r_mag * np.sin(TA), 0])
        cos_RAAN, sin_RAAN = np.cos(RAAN_t), np.sin(RAAN_t)
        cos_inc, sin_inc = np.cos(inc), np.sin(inc)
        cos_AOP, sin_AOP = np.cos(AOP_t), np.sin(AOP_t)
        R1 = np.array([[cos_RAAN, -sin_RAAN, 0], [sin_RAAN, cos_RAAN, 0], [0, 0, 1]])
        R2 = np.array([[1, 0, 0], [0, cos_inc, -sin_inc], [0, sin_inc, cos_inc]])
        R3 = np.array([[cos_AOP, -sin_AOP, 0], [sin_AOP, cos_AOP, 0], [0, 0, 1]])
        r_eci = R1 @ R2 @ R3 @ r_pf
        positions.append(r_eci)
        times.append(EPOCH + timedelta(seconds=int(t)))
    return np.array(positions), np.array(times)

def get_daytime_passes(all_passes):
    daytime_passes = []
    kst_tz = timezone(kst_offset)
    for p_start_naive, p_end_naive in all_passes:
        p_start_utc = p_start_naive.replace(tzinfo=timezone.utc)
        p_end_utc = p_end_naive.replace(tzinfo=timezone.utc)
        pass_date_kst = p_start_utc.astimezone(kst_tz)
        pass_date_str = pass_date_kst.strftime('%Y-%m-%d')
        if pass_date_str in daily_sun_times_kst:
            sunrise_kst_t, sunset_kst_t = daily_sun_times_kst[pass_date_str]
            sunrise_dt_kst = datetime.combine(pass_date_kst.date(), sunrise_kst_t, tzinfo=kst_tz)
            sunset_dt_kst = datetime.combine(pass_date_kst.date(), sunset_kst_t, tzinfo=kst_tz)
            if p_start_utc < sunset_dt_kst and p_end_utc > sunrise_dt_kst:
                daytime_passes.append((p_start_naive, p_end_naive))
    return daytime_passes, len(daytime_passes)

def find_passes(a, e, inc, RAAN, AOP, M0, duration_sec, dt_sec):
    pos, times = propagate_orbit(a, e, inc, RAAN, AOP, M0, duration_sec, dt_sec)
    if pos is None or len(pos) == 0: return 0, 0, 0, []
    all_pass_times, pass_durations, current_pass = [], [], []
    r_tgt_ecef = np.array([R_EARTH*np.cos(TARGET_LAT)*np.cos(TARGET_LON), R_EARTH*np.cos(TARGET_LAT)*np.sin(TARGET_LON), R_EARTH*np.sin(TARGET_LAT)])
    for i, r_eci in enumerate(pos):
        t = i * dt_sec
        gst = OMEGA_EARTH * t
        x_ecef_sat = r_eci[0]*np.cos(gst) + r_eci[1]*np.sin(gst)
        y_ecef_sat = -r_eci[0]*np.sin(gst) + r_eci[1]*np.cos(gst)
        r_sat_ecef = np.array([x_ecef_sat, y_ecef_sat, r_eci[2]])
        r_los_ecef = r_sat_ecef - r_tgt_ecef
        elevation = np.arcsin(np.clip(np.dot(r_los_ecef, r_tgt_ecef) / (np.linalg.norm(r_los_ecef) * np.linalg.norm(r_tgt_ecef)), -1.0, 1.0))
        if elevation >= MIN_ELEVATION:
            current_pass.append(times[i])
        else:
            if current_pass:
                all_pass_times.append((current_pass[0], current_pass[-1]))
                pass_durations.append((current_pass[-1] - current_pass[0]).total_seconds())
                current_pass = []
    if current_pass:
        all_pass_times.append((current_pass[0], current_pass[-1]))
        pass_durations.append((current_pass[-1] - current_pass[0]).total_seconds())
    return max(pass_durations, default=0), sum(pass_durations), len(all_pass_times), all_pass_times

# ==============================================================================
# 3. MAIN SWEEP EXECUTION
# ==============================================================================

altitude_km = 685
a = R_EARTH + altitude_km * 1e3
inc_rad = inclination_for_sso(a)
inc_deg = np.degrees(inc_rad)
e = 0.0001
print(f"SSO Parameters: Alt={altitude_km}km, Inc={inc_deg:.3f}° (Fixed)")

raan_sweep_values = range(0, 360, 60)
aop_sweep_values = [0]
m0_sweep_values = range(0, 360, 60)

sso_all_results_list = []
total_sims = len(raan_sweep_values) * len(aop_sweep_values) * len(m0_sweep_values)
completed_sims = 0
print(f"Starting SSO sweep for {total_sims} combinations (7-Day Sim)...")
sweep_start_time = py_time.time()

for RAAN_deg in raan_sweep_values:
    RAAN_rad = np.radians(RAAN_deg)
    for AOP_deg in aop_sweep_values:
        AOP_rad = np.radians(AOP_deg)
        for M0_deg in m0_sweep_values:
            M0_rad = np.radians(M0_deg)
            completed_sims += 1
            if completed_sims % 10 == 0:
                print(f"   Running simulation {completed_sims}/{total_sims}...")
            result_dict = {"altitude_km": altitude_km, "inclination_deg": inc_deg, "RAAN_deg": RAAN_deg, "AOP_deg": AOP_deg, "M0_deg": M0_deg, "max_pass_duration_s": 0, "total_daytime_duration_s": 0, "num_passes": 0, "num_daytime_passes": 0, "all_pass_times_utc_tuples": []}
            try:
                _, _, num_p, all_pass_times = find_passes(a, e, inc_rad, RAAN_rad, AOP_rad, M0_rad, duration_sec=SIM_DURATION, dt_sec=TIME_STEP)
                result_dict["num_passes"] = num_p
                result_dict["all_pass_times_utc_tuples"] = str(all_pass_times)
                if all_pass_times:
                    daytime_passes, num_daytime_passes = get_daytime_passes(all_pass_times)
                    result_dict["num_daytime_passes"] = num_daytime_passes
                    if num_daytime_passes > 0:
                        daytime_durations = [(p_end - p_start).total_seconds() for p_start, p_end in daytime_passes]
                        result_dict["max_pass_duration_s"] = max(daytime_durations)
                        result_dict["total_daytime_duration_s"] = sum(daytime_durations)
            except Exception as e:
                print(f"   ❌ Error for RAAN={RAAN_deg}, M0={M0_deg}: {e}")
            sso_all_results_list.append(result_dict)

print(f"SSO sweep complete. Time taken: {py_time.time() - sweep_start_time:.2f} seconds.")

if not sso_all_results_list:
    print("❌ Error: No results generated from SSO sweep.")
else:
    df_sso_all = pd.DataFrame(sso_all_results_list)
    try:
        df_sso_all.to_csv(output_path, index=False)
        print(f"\n✅ Saved ALL {len(df_sso_all)} raw sweep results to: {output_path}")
    except Exception as e:
        print(f"❌ Error saving all raw SSO results: {e}")
print("\n--- SSO 7-Day Data Generation Script Finished ---")


--- Starting Comprehensive SSO Sweep (7-Day Data Generation) ---
Output will be saved to: /content/drive/MyDrive/ThesisData/All_SSO_Sweep_Raw_Results_7day.csv
Using daily KST sunrise/sunset times for 8 days.
SSO Parameters: Alt=685km, Inc=98.116° (Fixed)
Starting SSO sweep for 36 combinations (7-Day Sim)...
   Running simulation 10/36...
   Running simulation 20/36...
   Running simulation 30/36...
SSO sweep complete. Time taken: 102.33 seconds.

✅ Saved ALL 36 raw sweep results to: /content/drive/MyDrive/ThesisData/All_SSO_Sweep_Raw_Results_7day.csv

--- SSO 7-Day Data Generation Script Finished ---


In [ ]:
# === Global SSO Ranking Analysis (7-Day) ===
# Description: This script loads the 7-day raw data from the SSO sweep, calculates a
#              global composite score, and ranks all simulated configurations.

import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

print("--- Starting Global SSO Ranking Analysis (7-Day) ---")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================

# --- File Paths ---
input_dir = '/content/drive/MyDrive/ThesisData'
input_filename = "All_SSO_Sweep_Raw_Results_7day.csv"
input_path = os.path.join(input_dir, input_filename)
output_filename = "All_SSO_Ranked_Results_7day.csv"
output_path = os.path.join(input_dir, output_filename)

# --- Scoring Configuration ---
metrics_to_normalize = [
    'max_pass_duration_s',
    'total_daytime_duration_s',
    'num_daytime_passes'
]
# You can adjust these weights as needed for your analysis.
weights = {
    'max_pass_duration_s': 0.3,
    'total_daytime_duration_s': 0.3,
    'num_daytime_passes': 0.4
}
print(f"Loading raw data from: {input_path}")
print(f"Metrics for scoring: {metrics_to_normalize}")
print(f"Weights being used: {weights}")

# ==============================================================================
# 2. ANALYSIS EXECUTION
# ==============================================================================

try:
    df_all_results = pd.read_csv(input_path)
    print(f"\nSuccessfully loaded {len(df_all_results)} simulation configurations.")

    if df_all_results[metrics_to_normalize].sum().sum() == 0:
        print("\n⚠️ Warning: All scoring metrics are zero. Cannot calculate a meaningful score.")
    else:
        print("\nNormalizing metrics across the entire dataset...")
        scaler = MinMaxScaler()
        norm_cols = [f"{col}_norm" for col in metrics_to_normalize]
        df_all_results[norm_cols] = scaler.fit_transform(df_all_results[metrics_to_normalize])
        print("Normalization complete.")

        print("\nCalculating global composite score...")
        df_all_results['composite_score'] = (
            df_all_results['max_pass_duration_s_norm'] * weights['max_pass_duration_s'] +
            df_all_results['total_daytime_duration_s_norm'] * weights['total_daytime_duration_s'] +
            df_all_results['num_daytime_passes_norm'] * weights['num_daytime_passes']
        )
        print("Score calculation complete.")

        print("\nRanking all configurations by composite score...")
        df_ranked = df_all_results.sort_values(by='composite_score', ascending=False).reset_index(drop=True)

        print("\n" + "="*80)
        print("=== TOP 5 BEST PERFORMING SSO CONFIGURATIONS (7-DAY GLOBAL RANKING) ===")
        print("="*80)
        display_cols = ['RAAN_deg', 'AOP_deg', 'M0_deg', 'composite_score', 'max_pass_duration_s', 'total_daytime_duration_s', 'num_daytime_passes']
        print(df_ranked.head(5)[display_cols].to_string())
        print("="*80)

        try:
            df_ranked.to_csv(output_path, index=False)
            print(f"\n✅ Successfully saved the full ranked results to: {output_path}")
        except Exception as e:
            print(f"\n❌ Error saving ranked results file: {e}")

except FileNotFoundError:
    print(f"\n❌ Error: The input file was not found at '{input_path}'.")
except Exception as e:
    print(f"\nAn unexpected error occurred during analysis: {e}")

print("\n--- Global SSO Analysis (7-Day) Script Finished ---")


--- Starting Global SSO Ranking Analysis (7-Day) ---
Loading raw data from: /content/drive/MyDrive/ThesisData/All_SSO_Sweep_Raw_Results_7day.csv
Metrics for scoring: ['max_pass_duration_s', 'total_daytime_duration_s', 'num_daytime_passes']
Weights being used: {'max_pass_duration_s': 0.3, 'total_daytime_duration_s': 0.3, 'num_daytime_passes': 0.4}

Successfully loaded 36 simulation configurations.

Normalizing metrics across the entire dataset...
Normalization complete.

Calculating global composite score...
Score calculation complete.

Ranking all configurations by composite score...

=== TOP 5 BEST PERFORMING SSO CONFIGURATIONS (7-DAY GLOBAL RANKING) ===
   RAAN_deg  AOP_deg  M0_deg  composite_score  max_pass_duration_s  total_daytime_duration_s  num_daytime_passes
0       300        0     120         1.000000                380.0                    3030.0                  10
1       300        0      60         0.987500                380.0                    3000.0                  